In [47]:
import pandas as pd

In [48]:
bu = pd.read_csv("yelp_business.csv")

In [49]:
# Removes all values that are not in Santa Barbara
bu = bu[bu["city"] == "Santa Barbara"]

In [50]:
# Removes all values that are not restaurants
bu["categories"] = bu["categories"].astype("str")
bu["categories"] = bu["categories"].apply(lambda x: set(x.split(", ")))

# Creates a temporary column, performs the intersection operation on the word_list and the temporary column, and removes any 
# values that are empty in the temp column
word_list = ["Restaurants", "Food", "Restaurant"]
bu["temp_categories"] = bu["categories"].apply(lambda x: x.intersection(word_list))
bu = bu[bu["temp_categories"] != set()]
bu = bu.drop(['temp_categories'], axis=1)

In [51]:
# Removes all values that are not open
bu = bu[bu["is_open"] == 1]

In [52]:
# Drops unnessescary columns and does some formatting 
bu = bu.drop(columns=['_id', 'city', 'is_open', 'state'])
bu["attributes.RestaurantsTakeOut"].fillna("False", inplace = True)
bu["attributes.RestaurantsTakeOut"] = bu["attributes.RestaurantsTakeOut"].replace("None", "False")
bu = bu.rename(columns={"attributes.RestaurantsTakeOut":"take_out"})

In [53]:
# Reads the reviews csv
re = pd.read_csv("yelp_review.csv")

In [54]:
# Make the st series which is indexed by tuples of business id and star value eg ("thisisanid", 1)
re = pd.merge(bu, re, how ='inner', on =['business_id']).drop(columns=["address", "take_out", "latitude", "longitude", "name", 
                                                               "review_count", "stars_x"]).rename(columns={"stars_y":"stars"})
st = re.groupby(["business_id", "stars"]).count()['review_id']

In [55]:
# Iterates through st and makes lists of star values, and adds them to the dictionary with the business id as the code
# stars["thisisanid"] = [0,0,0,10,50] is a for a business with id "thisisanid" that got 0 1,2, and 3 star reviews, 10 4 star and 50 5 star
stars = {}
for key in st.index:
    if key[0] not in stars:
        stars[key[0]] = [0,0,0,0,0]

    stars[key[0]][key[1]-1] = st[key]

In [56]:
# Gets the dates for each restaurant/star value combo
dt = re.groupby(["business_id", "stars"]).max()["date"]
dt.head()

business_id             stars
-3AooxIkg38UyUdlz5oXdw  1        2021-12-28 02:40:57
                        2        2021-11-29 16:04:16
                        3        2021-10-01 02:33:33
                        4        2021-12-29 02:04:15
                        5        2022-01-05 05:43:43
Name: date, dtype: object

In [57]:
# Does the same thing as above with the dates
dates = {}
for key in dt.index:
    if key[0] not in dates:
        dates[key[0]] = [0,0,0,0,0]

    dates[key[0]][key[1]-1] = dt[key]

In [58]:
# sorts bu
bu = bu.sort_values(by='business_id')

In [59]:
# adds star/date lists to bu
bu['star_count'] = stars.values()
bu['dates'] = dates.values()

In [60]:
# writes bu to the csv
bu.to_csv('yelp_business_clean.csv', index=False)

In [61]:
bu.head()

,address,take_out,business_id,categories,latitude,longitude,name,review_count,stars,star_count,dates
9418,1012 State St,True,-3AooxIkg38UyUdlz5oXdw,"{Italian, Nightlife, Event Planning & Services...",34.421931,-119.702185,Chase Restaurant,436,3.0,"[115, 56, 67, 62, 144]","[2021-12-28 02:40:57, 2021-11-29 16:04:16, 202..."
76606,2036 Cliff Dr,True,-6jvfSJGprbfBD2QrS9zQw,"{Food, Grocery}",34.402538,-119.724894,Mesa Produce,19,5.0,"[0, 0, 0, 0, 19]","[0, 0, 0, 0, 2021-10-24 16:08:48]"
62010,1417 San Andres St,True,-ALqLSTzkGDMscHdxA1NgA,"{Mexican, Restaurants}",34.418221,-119.715795,Su Casa Fresh Mexican Grill,29,4.5,"[0, 2, 3, 7, 17]","[0, 2013-01-24 23:23:09, 2016-08-13 05:59:35, ..."
134706,1905 Cliff Dr,True,-BdYhP-12elmFV7oB1iv4A,"{Gastropubs, Nightlife, American (New), Cockta...",34.401382,-119.722472,Corner Tap Room,41,5.0,"[0, 0, 2, 5, 37]","[0, 0, 2021-10-21 03:56:27, 2021-10-30 15:46:1..."
92579,31 W Carrillo St,True,-FM4CxOg4XXmX_Ebky_SiQ,"{Breakfast & Brunch, Nightlife, American (New)...",34.420361,-119.702475,Finch & Fork,1405,4.0,"[63, 82, 132, 379, 778]","[2021-11-21 02:44:59, 2021-06-21 00:01:19, 202..."
